# Inference

## Dependencies and imports

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM

## Load fintuned model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("VerbACxSS/sempl-it-gpt2-small-italian", model_max_length=1024)
model = AutoModelForCausalLM.from_pretrained("VerbACxSS/sempl-it-gpt2-small-italian")

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

model.eval().to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(30001, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=30001, bias=False)
)

## Load datasets

In [5]:
train_df = pd.read_csv('../dataset/train.tsv', sep='\t', encoding='utf-8')
test_df = pd.read_csv('../dataset/test.tsv', sep='\t', encoding='utf-8')
val_df = pd.read_csv('../dataset/val.tsv', sep='\t', encoding='utf-8')

## Utilities

In [6]:
TOKENIZER_MAX_LENGTH = 1024

def predict(samples_array, batch_size=16):
  prompts = [f'### [Input]:\n{s}\n\n###[Output]:\n' for s in samples_array]
  batched_prompt = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

  outputs = []
  for batch in tqdm(batched_prompt):
    x = tokenizer(batch, max_length=TOKENIZER_MAX_LENGTH, truncation=True, padding=True, return_tensors='pt').input_ids.to('cuda')
    y = model.generate(x, max_length=TOKENIZER_MAX_LENGTH)
    y_decs = tokenizer.batch_decode(y, max_length=TOKENIZER_MAX_LENGTH, truncation=True)
    y_decs = [y_dec.split("###[Output]:\n")[1].split('<|endoftext|>')[0].strip() for y_dec in y_decs]
    outputs.extend(y_decs)
  return outputs

## Random predictions

In [7]:
for s in test_df.sample(10).to_dict(orient='records'):
  output = predict([s['original_text']])[0]
  print("\noriginal: ", s['original_text'])
  print("\noracle: ", s['oracle_text'])
  print("\nmodel: ", output)
  print('----------------')

100%|██████████| 1/1 [00:08<00:00,  8.86s/it]



original:  Con la prima Direttiva per la trasparenza dei documenti di fatturazione del Servizio Idrico Integrato, di cui alla deliberazione ARERA 586/2012/R/IDR, l'ARERA, al fine di rendere le bollette più comprensibili e trasparenti, ha introdotto uno schema di bolletta più esaustivo e più chiaro, corredato di spiegazioni.

oracle:  Con la prima Direttiva per la trasparenza dei documenti di fatturazione del Servizio Idrico Integrato, l'ARERA ha introdotto un nuovo schema di bolletta che rende le bollette più chiare e facili da capire.

model:  Con la prima Direttiva per la trasparenza dei documenti di fatturazione del Servizio Idrico Integrato, che è stata approvata con la deliberazione ARERA 586/2012/R/IDR, l'ARERA ha introdotto un nuovo schema di bolletta più chiaro e più chiaro, accompagnato da spiegazioni.
----------------


100%|██████████| 1/1 [00:01<00:00,  1.05s/it]



original:  Ad oggi la compagine societaria di ASA SpA è costituita dai Comuni della Conferenza Territoriale n°5 "Toscana Costa" e da IRETI SpA (soggetto che ha incorporato AGA S.p.A detentrice della quota ceduta a seguito di procedura di gara ad evidenza pubblica avvenuta nel Giugno2004)

oracle:  Attualmente, ASA SpA è posseduta dai Comuni della Conferenza Territoriale n°5 "Toscana Costa" e da IRETI SpA. Quest'ultima ha acquisito la partecipazione precedentemente detenuta da AGA S.p.A attraverso un'asta pubblica nel giugno 2004.

model:  Attualmente, la società ASA SpA è formata dai Comuni della Conferenza Territoriale n°5 "Toscana Costa" e da IRETI SpA, che ha incorporato AGA S.p.A nella società AGA S.p.A. dopo la procedura di gara pubblica avvenuta nel Giugno 2004.
----------------


100%|██████████| 1/1 [00:01<00:00,  1.36s/it]


...


original:  Le modalità di eleggibilità dei pazienti per i percorsi di assistenza e la successiva presa in carico attraverso una valutazione multidisciplinare e multiprofessionale delle condizioni e dei bisogni della persona, premessa indispensabile per definire l'attuazione di politiche e di interventi mirati ed efficaci;

oracle:  Come vengono scelti i pazienti per ricevere cure mediche e come vengono valutate le loro condizioni e i loro bisogni da un team di esperti. Questo è importante per decidere quali cure e interventi sono più adatti ed efficaci per loro.

model:  Le persone possono essere eleggibili per ricevere assistenza e cure mediche, e poi si possono valutare le condizioni e i bisogni della persona in modo completo. Questo è importante per definire politiche e interventi mirati e efficaci.
----------------


100%|██████████| 1/1 [00:00<00:00,  4.90it/s]



original:  Esercizio di funzioni di indirizzo e monitoraggio sulla tematica della sicurezza delle cure;

oracle:  Esercizio su come gestire e controllare la sicurezza delle cure mediche.

model:  Attività per controllare e monitorare la sicurezza delle cure mediche.
----------------


100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


original:  L'organizzazione ha individuato le aree e i processi a rischio. In particolare, sono individuati e gestiti i processi relativi a:
- la corretta identificazione del paziente;
- l'esecuzione delle principali manovre strumentali previste;
- la corretta gestione dei dispositivi medici durante le attività;
- la gestione di situazioni d'emergenza clinica e il trasferimento del paziente in idonea struttura;
- la corretta raccolta, gestione e trasporto sicuro dei campioni di laboratorio

oracle:  L'organizzazione ha individuato le aree e i processi a rischio, come ad esempio l'identificazione del paziente, l'uso degli strumenti medici, la gestione dei dispositivi durante le attività, l'intervento in situazioni di emergenza e il trasferimento del paziente in una struttura adeguata, e infine la raccolta, gestione e trasporto sicuro dei campioni di laboratorio.

model:  L'organizzazione ha identificato le aree e i processi a rischio. In particolare, si occupa di:
- identificare corret

## Run all predictions

In [9]:
train_df['model_text'] = predict(train_df['original_text'].tolist(), batch_size=1)
test_df['model_text'] = predict(test_df['original_text'].tolist(), batch_size=1)
val_df['model_text'] = predict(val_df['original_text'].tolist(), batch_size=1)

100%|██████████| 1835/1835 [20:07<00:00,  1.52it/s]


## Save simplified datasets

In [10]:
train_df.to_csv('./train_simplified.tsv', index=False, sep='\t')
test_df.to_csv('./test_simplified.tsv', index=False, sep='\t')
val_df.to_csv('./val_simplified.tsv', index=False, sep='\t')